In [1]:
%pip install langchain langchain_community faiss-cpu langchain-groq dotenv langchain-huggingface

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
import json
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
COMPANY_DATA_FILE = "normalized_data.json"
COMPANY_FAISS_INDEX_DIR = "my_faiss_index_company_chunked"
GENERAL_QA_FAISS_INDEX_DIR = "my_faiss_index_general_qa_chunked"
LLM_MODEL_ID = "llama-3.1-8b-instant"
GENERAL_QA_DATA_FILE = "general_qa_data.json"
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [ ]:
company_vectorstore = None
general_qa_vectorstore = None
embeddings = None
llm_model = None

In [ ]:
# --- 1. Data Loading and Text Splitting ---
def load_and_chunk_data(file_path):
    """Loads JSON data and splits it into LangChain Documents."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            raw_data = json.load(f)
        print(f"Data: JSON data loaded successfully from '{file_path}'.")
    except FileNotFoundError:
        print(
            f"Error: '{file_path}' not found. This should have been handled by dummy data creation or manual upload. Returning empty list."
        )
        return []
    except json.JSONDecodeError:
        print(
            f"Error: Could not decode JSON from '{file_path}'. Please check file format. Returning empty list."
        )
        return []
    except Exception as e:
        print(f"Error loading JSON data from '{file_path}': {e}. Returning empty list.")
        return []

    initial_documents = []
    if not raw_data:  # Handle empty JSON file
        print(f"Data: '{file_path}' is empty. No documents to process.")
        return []

    for url, text_content in raw_data.items():
        # Ensure text_content is a string, as sometimes JSON values can be other types
        if isinstance(text_content, str):
            initial_documents.append(
                Document(page_content=text_content, metadata={"source": url})
            )
        else:
            print(f"Data: Skipping non-string content for URL: {url}")

    if not initial_documents:
        print(f"Data: No valid documents extracted from '{file_path}'.")
        return []

    print(
        f"Data: Created {len(initial_documents)} initial LangChain Document objects for '{file_path}'."
    )

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    print(
        f"Text Splitter: Initialized RecursiveCharacterTextSplitter for '{file_path}'."
    )

    chunked_documents = text_splitter.split_documents(initial_documents)
    if not chunked_documents:
        print(f"Text Splitter: No chunks generated from '{file_path}'.")
        return []

    print(
        f"Text Splitter: Split {len(initial_documents)} documents into {len(chunked_documents)} chunks for '{file_path}'."
    )
    return chunked_documents

In [ ]:
# --- 2. Initialize Embeddings and FAISS ---
def initialize_vector_store(chunked_docs, faiss_index_dir):
    """Initializes embeddings and creates/loads FAISS vector store.

    This function takes chunked documents and a specified directory name (faiss_index_dir)
    to create or load the FAISS index. It handles creating the directory if it doesn't
    exist, ensuring separate storage for different datasets (like company and General Q&A).
    """
    global embeddings

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    print(f"Initialized HuggingFaceEmbeddings with model: {embeddings.model_name}")

    if not os.path.exists(faiss_index_dir):
        os.makedirs(faiss_index_dir)

    vectorstore = None
    index_file_path = os.path.join(faiss_index_dir, "index.faiss")

    if (
        os.path.exists(faiss_index_dir)
        and os.path.isdir(faiss_index_dir)
        and os.path.exists(index_file_path)
    ):
        try:
            vectorstore = FAISS.load_local(
                faiss_index_dir, embeddings, allow_dangerous_deserialization=True
            )
            print(f"FAISS: Index loaded successfully from '{faiss_index_dir}'.")
        except Exception as e:
            print(
                f"FAISS: Error loading existing index from '{faiss_index_dir}': {e}. Attempting to recreate."
            )
            try:
                vectorstore = FAISS.from_documents(chunked_docs, embeddings)
                vectorstore.save_local(faiss_index_dir)
                print(f"FAISS: Index recreated and saved to '{faiss_index_dir}'.")
            except Exception as create_e:
                print(
                    f"FAISS: Failed to recreate index for '{faiss_index_dir}': {create_e}. Returning None."
                )
                return None
    else:
        print(
            f"FAISS: Directory or index '{faiss_index_dir}' not found. Creating new index..."
        )
        try:
            vectorstore = FAISS.from_documents(chunked_docs, embeddings)
            vectorstore.save_local(faiss_index_dir)
            print(f"FAISS: New index created and saved to '{faiss_index_dir}'.")
        except Exception as create_e:
            print(
                f"FAISS: Failed to create new index for '{faiss_index_dir}': {create_e}. Returning None."
            )
            return None

    if vectorstore is None:
        print(
            f"FAISS: Vector store for '{faiss_index_dir}' is None after initialization attempt. This indicates a problem."
        )

    return vectorstore

In [ ]:
def initialize_llm():
    """Initializes the tokenizer, model, and pipeline for the open-source LLM."""
    global llm_model

    # print(f" LLM: Initializing Groq ChatModel with model: {LLM_MODEL_ID}...")
    try:
        llm_model = ChatGroq(
            model=LLM_MODEL_ID,
            temperature=0.7,
            groq_api_key=GROQ_API_KEY,
        )
    except Exception as e:
        # print(f"LLM: Error initializing Groq ChatModel '{LLM_MODEL_ID}': {e}")
        # print("Please ensure your GROQ_API_KEY is set correctly and you have internet access.")
        llm_model = None
    return llm_model

In [ ]:
# --- Intent Classification Function ---
def classify_question_intent(question, llm):
    """
    Classifies the user's question as 'COMPANY', 'GENERAL', or 'OTHER'.
    Uses a strict prompt to guide the LLM's classification.
    """
    classification_prompt = f"""
    You are an expert question classifier. Classify the following user question into one of these categories: 'COMPANY', 'GENERAL', 'OTHER'.
    - 'COMPANY': Questions specifically about company, their services, products, projects, team, or any business-related aspects of company. This includes follow-up questions about previous company topics.
    - 'GENERAL': Common conversational questions like greetings, how are you, current time, general knowledge (e.g., "what is AI?"), or anything that is NOT directly about company.
    - 'OTHER': Any question that does not clearly fit into 'company' or 'GENERAL' categories, or is entirely out of scope.

    Respond with ONLY one word, uppercase: 'COMPANY', 'GENERAL', or 'OTHER'.
    Do not add any other text, punctuation, or explanation.

    Examples:
    Question: "Hi"
    Category: GENERAL

    Question: "How are you?"
    Category: GENERAL

    Question: "What's up?"
    Category: GENERAL

    Question: "Tell me about company."
    Category: company

    Question: "What services do they offer?"
    Category: company

    Question: "How are you doing today?"
    Category: GENERAL

    Question: "What is the capital of France?"
    Category: OTHER

    Question: "{question}"
    Category:
    """
    messages = [
        SystemMessage(
            content="You are a classifier assistant. Respond ONLY with one of the specified categories: 'company', 'GENERAL', 'OTHER'."
        ),
        HumanMessage(content=classification_prompt),
    ]
    try:
        response = llm.invoke(messages)
        classification = response.content.strip().upper()
        # Basic validation for classification output
        if classification in ["company", "GENERAL", "OTHER"]:
            print(f"Intent Classifier: Question classified as '{classification}'.")
            return classification
        else:
            print(
                f"Intent Classifier: Unexpected classification output: '{classification}'. Defaulting to 'OTHER'."
            )
            return "OTHER"
    except Exception as e:
        print(
            f"Intent Classifier: Error during classification: {e}. Defaulting to 'OTHER'."
        )
        return "OTHER"

In [ ]:
# --- Main Chatbot Logic ---
def run_chatbot():
    """Main function to run the command-line chatbot with dual knowledge bases."""
    global company_vectorstore, general_qa_vectorstore

    print("\n--- Initializing Chatbot Components ---")

    # Load and chunk company data, then initialize its vector store
    company_chunked_documents = load_and_chunk_data(COMPANY_DATA_FILE)
    if not company_chunked_documents:
        print("Exiting: No company data to process or chunking failed.")
        return
    # Use the initialize_vector_store function to create/load the company vector store
    company_vectorstore = initialize_vector_store(
        company_chunked_documents, COMPANY_FAISS_INDEX_DIR
    )
    if company_vectorstore is None:
        print(
            "Exiting: Failed to initialize company FAISS vector store. Please check errors above."
        )
        return
    print(
        f"Global: company_vectorstore is {'initialized' if company_vectorstore else 'NOT initialized'}."
    )

    # Load and chunk General Q&A data, then initialize its vector store
    general_qa_chunked_documents = load_and_chunk_data(GENERAL_QA_DATA_FILE)
    if not general_qa_chunked_documents:
        print("Exiting: No General Q&A data to process or chunking failed.")
        return
    # Use the initialize_vector_store function to create/load the General Q&A vector store
    general_qa_vectorstore = initialize_vector_store(
        general_qa_chunked_documents, GENERAL_QA_FAISS_INDEX_DIR
    )
    if general_qa_vectorstore is None:
        print(
            "Exiting: Failed to initialize General Q&A FAISS vector store. Please check errors above."
        )
        return
    print(
        f"Global: general_qa_vectorstore is {'initialized' if general_qa_vectorstore else 'NOT initialized'}."
    )

    # Initialize LLM (Groq)
    llm_model = initialize_llm()
    if llm_model is None:
        print("Exiting: Failed to initialize LLM. Please check errors above.")
        return
    print(f"Global: llm_model is {'initialized' if llm_model else 'NOT initialized'}.")

    print("\n--- Chatbot Ready! Type 'exit' to quit ---")
    # Initialize chat history for the session
    chat_history_messages = []
    while True:
        user_question = input("\n🧑 You: ").strip()
        if user_question.lower() == "exit":
            print("🤖 Chatbot: Goodbye!")
            break

        if not user_question:
            print("🤖 Chatbot: Please type a question.")
            continue

        print("🤖 Chatbot: Thinking...")
        try:
            # Step 1: Classify the user's intent
            # Ensure LLM is available for classification
            if llm_model is None:
                print("Chatbot: LLM is not initialized. Cannot classify intent.")
                llm_answer = "I'm sorry, my core components are not ready. Please restart the chatbot."
                print(f"🤖 Chatbot: {llm_answer}")
                continue

            intent = classify_question_intent(user_question, llm_model)

            retrieved_docs = []
            context = ""
            current_llm_answer = ""
            # Step 2: Perform retrieval based on intent
            if intent == "company":
                print("Router: Detected 'company' intent.")
                if company_vectorstore:
                    retrieved_docs = company_vectorstore.similarity_search(
                        user_question, k=6
                    )
                else:
                    print(
                        "Router: company vector store not initialized. Cannot retrieve data."
                    )
            elif intent == "GENERAL":
                print("Router: Detected 'GENERAL' intent.")
                if general_qa_vectorstore:
                    retrieved_docs = general_qa_vectorstore.similarity_search(
                        user_question, k=5
                    )
                else:
                    print(
                        "Router: General QA vector store not initialized. Cannot retrieve data."
                    )
            else:  # intent == 'OTHER'
                print("Router: Detected 'OTHER' intent or classification failed.")
                llm_answer = "I don't know. My knowledge is limited to company and general questions. Please try rephrasing or ask a different question."
                print(f"🤖 Chatbot: {llm_answer}")
                # Add current user and assistant message to history before continuing
                chat_history_messages.append(HumanMessage(content=user_question))
                chat_history_messages.append(AIMessage(content=current_llm_answer))
                continue

            context = "\n\n".join([doc.page_content for doc in retrieved_docs])
            print(
                f"Retrieved {len(retrieved_docs)} chunks from the relevant FAISS index."
            )

            # Step 3: Construct the prompt for the LLM
            messages = [
                SystemMessage(
                    content="You are a helpful and friendly assistant. Answer the user's question based on the provided context. If the information is not present in the context, or if you cannot provide a direct and accurate answer based solely on the context, respond with 'I don't know.' Do not invent information. For general conversational queries, try to give a friendly response even if the information is brief."
                ),
            ]
            messages.extend(chat_history_messages)  # Add previous turns
            messages.append(
                HumanMessage(
                    content=f"Context:\n{context}\n\nQuestion: {user_question}"
                )
            )

            # Step 4: Generate response using the Groq LLM
            llm_response = llm_model.invoke(messages)
            llm_answer = llm_response.content.strip()

            # Step 5: Post-processing to enforce "I don't know" strictly

            if not context.strip() or len(retrieved_docs) == 0:
                llm_answer = "I don't know. The information to answer your question was not found in my knowledge base."
            else:
                # If LLM gave an answer, and we had context, we assume it's good.
                pass

            print(f"🤖 Chatbot: {llm_answer}")
            # Update chat history for the next turn with the original user question and the final answer
            chat_history_messages.append(HumanMessage(content=user_question))
            chat_history_messages.append(AIMessage(content=current_llm_answer))
        except Exception as e:
            print(f"An error occurred during chat processing: {e}")
            print(
                "🤖 Chatbot: I apologize, but something went wrong. Please try again."
            )

In [ ]:
if __name__ == "__main__":
    run_chatbot()